# Known Problem:
- **Unresolved**
    1. how to solve for eigen vectors
- **Partially Resolved**
    1. Certain websites cause the error of, specified here: https://stackoverflow.com/questions/23013220/max-retries-exceeded-with-url-in-requests, which is a connection error. 
        - this appears to be windows specific as this was not a relevant issue when testing code in ubuntu 22.04, some forum reflect similar message
        - FAILED SOLUTION: first response in that stackoverflow page, still kept the stuff bc it seemed like better QOL
        - ATTEMPTED SOLUTION: used try and except. NOTE: this solution is not ideal as it does not resolve the issue, it just ignores it and moves on.
- **Resolved**
    1. The search "what is a taylor series" returns SSLCertVerification Error.
        - this resolution does not consider the security of the websites being accessed, potentially use try except to have the request verify when possible and not break when cannot verify
        - ATTEMPTED SOLUTION: used try except to use verify=False when ssl error occurs. Nested try except to ensure that both connection error and ssl error are handled.

    

    

# Part 1: Data Acquisition & Preparation

## Collecting & Preprocessing Our Data

In [2]:
#imports
import requests #send HTTP requests to web pages and retrieve their content
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from urllib.parse import urlparse, urljoin
import pandas as pd
from nltk.corpus import stopwords
import http.client
import re

In [3]:
import ssl
#ssl._create_default_https_context = ssl._create_unverified_context
#uncomment if you cannot properly download punkt

nltk.download('punkt')
#ssl._create_default_https_context = ssl._create_default_https_context
#uncomment if you cannot properly download punkt

#remove filler words from an example column
nltk.download("stopwords")

nltk.download('wordnet')

[nltk_data] Downloading package punkt to /home/william/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/william/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/william/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
import concurrent.futures
from tqdm import tqdm
#Working implementation

session = requests.Session()
retry = Retry(connect=2, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)
# Define the search query and retrieve search results
search_query = input("What would you like to search up?")
search_engine_url = "https://www.google.com/search?q="

# Set headers to mimic a browser request
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# Number of search result pages to retrieve (each page typically has 10 results)
num_pages = 5

search_results = []

def verify_ssl_certificate(url):
    try:
        response = session.get(url, verify=True, headers=headers)
        return True
    except requests.exceptions.SSLError:
        return False
    except Exception as e:
        print("Error:", e)
        return False

for page in range(num_pages):
    # Calculate the start index for the current page
    start_index = page * 10

    try:
        # Make a GET request to retrieve the search result page
        response = session.get(search_engine_url + search_query + f"&start={start_index}", headers=headers)
        response.raise_for_status()  # Raise an exception for non-2xx status codes

    except requests.exceptions.RequestException as e:
        print("Error occurred for page", page+1)
        print("Exception:", e)
        continue

    # Parse and extract information from the search result page
    soup = BeautifulSoup(response.content, "html.parser")

    for result in soup.select("div.g"):
        title_element = result.select_one("h3")
        if title_element:
            title = title_element.get_text()
            url_element = result.find("a")
            if url_element:
                url = url_element["href"]
                parsed_url = urlparse(url)
                if parsed_url.scheme == "":
                    # Relative URL, convert it to absolute URL
                    base_url = response.url
                    url = urljoin(base_url, url)

                # Verify SSL certificate before making the GET request
                if verify_ssl_certificate(url):
                    try:
                        print("Trying:", url)
                        result_response = session.get(url, headers=headers)
                        result_response.raise_for_status()  # Raise an exception for non-2xx status codes
                        result_soup = BeautifulSoup(result_response.content, "html.parser")
                        text = result_soup.get_text()
                        text = ' '.join(text.split())
                        search_results.append({"Title": title, "URL": url, "Text": text})
                    except http.client.IncompleteRead:
                        print("Error: Incomplete read for:", url)
                        continue
                    except Exception as e:
                        print("Error: Failed to retrieve content for:", url)
                        print("Exception:", e)
                        continue

                # Make a GET request to the URL of the search result
                else:
                    print("SSL certificate verification failed for:", url)
                    continue
                    

print("Grabbed search results")

# Process the search results as desired

Trying: https://www.tutorialspoint.com/deep_learning_with_keras/keras_creating_deep_learning_model.htm
Trying: https://www.tutorialspoint.com/deep_learning_with_keras/keras_creating_deep_learning_model.htm
Trying: https://www.digitalocean.com/community/tutorials/how-to-build-a-deep-learning-model-to-predict-employee-retention-using-keras-and-tensorflow
Trying: https://www.analyticsvidhya.com/blog/2021/05/develop-your-first-deep-learning-model-in-python-with-keras/
Trying: https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/
Trying: https://towardsdatascience.com/building-a-deep-learning-model-using-keras-1548ca149d37
Trying: https://www.seldon.io/how-to-build-a-machine-learning-model
Trying: https://www.simplilearn.com/tutorials/deep-learning-tutorial/deep-learning-with-python
Trying: https://www.educba.com/deep-learning-model/
Trying: https://www.codecademy.com/learn/paths/build-deep-learning-models-with-tensorflow
Trying: https://www.tensorflow.org/overview


In [5]:
df = pd.DataFrame(search_results)
df_bart = pd.DataFrame(search_results)

# Extract domain names from URLs and set as the index for the dataframe
tld_pattern = re.compile(r'\.[a-zA-Z]+$')
df['Domain'] = df['URL'].apply(lambda url: re.sub(tld_pattern, '', urlparse(url).netloc.replace("www.", "")))
df.insert(0, 'Domain', df.pop('Domain'))

df_bart['Domain'] = df_bart['URL'].apply(lambda url: re.sub(tld_pattern, '', urlparse(url).netloc.replace("www.", "")))
df_bart.insert(0, 'Domain', df_bart.pop('Domain'))
#Handles the "Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER" exception
df_bart['Text'] = df_bart['Text'].str.replace('\ufffd', '')

#unfiltered dataframe
print(len(df))
df.head()

48


,Domain,Title,URL,Text
0,tutorialspoint,Creating Deep Learning Model - Tutorialspoint,https://www.tutorialspoint.com/deep_learning_w...,Creating Deep Learning Model Home Coding Groun...
1,tutorialspoint,Creating Deep Learning Model - Tutorialspoint,https://www.tutorialspoint.com/deep_learning_w...,Creating Deep Learning Model Home Coding Groun...
2,digitalocean,How To Build a Deep Learning Model to Predict ...,https://www.digitalocean.com/community/tutoria...,How To Build a Deep Learning Model to Predict ...
3,analyticsvidhya,Develop your first Deep Learning Model in Pyth...,https://www.analyticsvidhya.com/blog/2021/05/d...,deep learning model in python with keras - Ana...
4,machinelearningmastery,Your First Deep Learning Project in Python wit...,https://machinelearningmastery.com/tutorial-fi...,Your First Deep Learning Project in Python wit...


In [6]:
stop_words = stopwords.words("english") #english filler words

def preprocess(text):
    '''
    Input a text block and filter out unneeded characters
    returns a filtered text block in the form of a str
    Function filters whitespace, numbers, special characters, stopwords; handles case normalization 
    '''
    # Remove special characters and numbers
    cleaned_text = re.sub(r"[^a-zA-Z]+", " ", text)
    
    # Convert to lowercase and split into words
    words = cleaned_text.lower().split()
    
    # Remove stop words and single-character words
    filtered_words = [word for word in words if word not in stop_words and len(word) > 1]
    lemmatized_text = " ".join([WordNetLemmatizer().lemmatize(word) for word in filtered_words])
    
    return lemmatized_text

def bart_preprocess(text):

    words = text.split()
    lemmatized_text = " ".join([WordNetLemmatizer().lemmatize(word) for word in words])

    return lemmatized_text


In [7]:
example = df["Text"][0]
processed_example = preprocess(example)

print("length of unfiltered text: ", len(example))
print("length of filtered text: ", len(processed_example))

length of unfiltered text:  4613
length of filtered text:  3426


In [8]:
#Properly using the apply() function onto a pandas column
#removes both filler words and special characters
df["Text"] = df["Text"].apply(preprocess)
df_bart["Text"] = df_bart["Text"].apply(bart_preprocess)


In [9]:
print(len(df["Text"][0]))

3426


In [10]:
#filtered text for first row entry
df["Text"][0]

'creating deep learning model home coding ground job whiteboard tool corporate training teach u login category academic tutorial big data analytics computer programming computer science database devops digital marketing engineering tutorial exam syllabus famous monument gate exam latest technology machine learning mainframe development management tutorial mathematics tutorial microsoft technology misc tutorial mobile development java technology python technology sap tutorial programming script selected reading software quality soft skill telecom tutorial upsc ia exam web development sport tutorial xml technology multi language interview question prime pack course ebooks library article login deep learning kera tutorial deep learning kera home deep learning kera introduction deep learning setting project importing library creating deep learning model compiling model preparing data training model evaluating model performance predicting test data saving model loading model prediction conc

# Part 2: EDA, Feature Engineering, and Model Development

### Future Optimizations

1. Ignore website navigation for websites like (wikipedia, stackoverflow, etc.)


### Tasks

1. Research into transformer, BART model, to better understand model set up and how to get highest performance
2. Experiment with BART input (lemmatize/not, capitalization, etc.) to find best performance 
3. Try to implement BART model with recursion as per the drawn diagram 
4. Research into paralellization of transformer models with HuggingFace 

### Notes

1. To actually make project work, we need to create a few metrics that we can train a DL model on. Maybe, linguistic complexity (formulaic), sentiment analysis, text length, etc.

In [11]:
import torch
from sklearn.model_selection import train_test_split

# Assuming you have a DataFrame called 'df' containing your data
# Split the data into train and test sets (80% train, 20% test)
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

# Split the train data further into train and validation sets (80% train, 20% validation)
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)

# Print the sizes of each dataset
print("Train set size:", len(train_data))
print("Validation set size:", len(val_data))
print("Test set size:", len(test_data))

Train set size: 30
Validation set size: 8
Test set size: 10


In [12]:
test_data.reset_index().drop(["index"], axis = 1)
val_data.reset_index().drop(["index"], axis = 1)
train_data.reset_index().drop(["index"], axis = 1)

,Domain,Title,URL,Text
0,frontiersin,An Introductory Review of Deep Learning for Pr...,https://www.frontiersin.org/articles/10.3389/f...,frontier introductory review deep learning pre...
1,forbes,The Five Ways To Build Machine Learning Models,https://www.forbes.com/sites/cognitiveworld/20...,five way build machine learning modelssubscrib...
2,towardsdatascience,Building A Deep Learning Model using Keras,https://towardsdatascience.com/building-a-deep...,building deep learning model using kera eijaz ...
3,turing,A Guide to Making Deep Learning Models General...,https://www.turing.com/kb/making-deep-learning...,guide making deep learning model generalize be...
4,stackoverflow,How to put machine learning models into produc...,https://stackoverflow.blog/2020/10/12/how-to-p...,put machine learning model production stack ov...
5,mathworks,Deep Learning in MATLAB - MATLAB & Simulink,https://www.mathworks.com/help/deeplearning/ug...,deep learning matlab matlab simulink skip cont...
6,spectrum.ieee,Here's How to Make Deep Learning More Sustainable,https://spectrum.ieee.org/deep-learning-sustai...,make deep learning sustainable ieee spectrumie...
7,codecademy,Build Deep Learning Models with TensorFlow,https://www.codecademy.com/learn/paths/build-d...,build deep learning model tensorflow codecadem...
8,databricks,What are Machine Learning Models?,https://www.databricks.com/glossary/machine-le...,machine learning model platformthe databricks ...
9,educba,Deep Learning Model | How to Create ...,https://www.educba.com/deep-learning-model/,deep learning model create deep learning model...


In [13]:
#Using Bart
# !pip install transformers
# !pip install torch
from transformers import BartForConditionalGeneration, AutoTokenizer

In [18]:
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")


NameError: name 'BartForConditionalGeneration' is not defined

In [15]:
# len(df_bart["Text"][0])

41231

In [20]:
# inputs = tokenizer([df_bart["Text"][2]],max_length = 1024,return_tensors="pt")


In [21]:
summary_ids = model.generate(inputs["input_ids"], num_beams=1, min_length=0, max_length=30)
tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

'What is RGB Color? – Nix Sensor Ltd – NIX Sensor Ltd Skip to content BRAND NEW Nix Mini 3'

In [19]:
from transformers import AutoTokenizer, LEDForConditionalGeneration

device = "cuda:0" if torch.cuda.is_available() else "cpu"

model = LEDForConditionalGeneration.from_pretrained("allenai/led-large-16384-arxiv")
tokenizer = AutoTokenizer.from_pretrained("allenai/led-large-16384-arxiv")


inputs = tokenizer.encode(df_bart["Text"][6], return_tensors="pt")

global_attention_mask = torch.zeros_like(inputs)
global_attention_mask[:, 0] = 1

model = model.to(device)
inputs = inputs.to(device)
global_attention_mask = global_attention_mask.to(device)

summary_ids = model.generate(inputs, global_attention_mask=global_attention_mask, num_beams = 1, max_length = 64)
print(tokenizer.decode(summary_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True))

 how to build a machine learning model is a key step in the process of embedding machine learning in your organisation. 
 machine learning model are powerful tool used to efficiently and effectively perform vital task and solve complex problems. 
 machine learning model are broadly split into three major types, supervised, unsupervised,
